In [152]:
%load_ext autoreload
%autoreload 2
import utils.dataset as dataset
import utils.preprocessing as preprocessing
from utils.logger import Logger
import lstm.model as model
import utils.postprocessing as postprocessing
import pandas as pd
from sklearn import metrics
from numpy import linalg as LA
import numpy as np
import logging
import argparse


from astropy.convolution import Gaussian1DKernel, convolve

import torch
import torch.nn as nn
from pathlib import Path
from torch.autograd import Variable

import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from sklearn.model_selection import KFold
import sklearn.preprocessing 

In [ ]:
cv = KFold(5, shuffle=False, random_state=0)

cv_list = []

for i, (idx_train, idx_test) in enumerate(cv.split(attack_data)):
    cv_list.append([idx_train, idx_test])

# SVM

In [27]:
from sklearn import svm

In [37]:
start_time = time.time()

In [38]:
print(start_time)

1553310354.434949


In [41]:
print(end_time)

1553310368.0227296


In [39]:
end_time = time.time()

In [42]:
print(end_time -start_time)

13.587780714035034


In [52]:
# comp_num = 45
clf = svm.OneClassSVM(nu=0.001, kernel="rbf", gamma=0.0001, verbose=True)
clf.fit(train_x[0:20000,0:2])
preds = clf.predict(test_x[5000:15000, 0:2]) 
unique, counts = np.unique(preds, return_counts=True)
print('predict 빈도:', dict(zip(unique, counts)))

[LibSVM]predict 빈도: {-1: 4842, 1: 5158}


In [46]:
# comp_num = 45
clf = svm.OneClassSVM(nu=0.001, kernel="rbf", gamma=0.0001, verbose=True)
clf.fit(train_x[0:20000,0:4])
preds = clf.predict(test_x[5000:15000, 0:4]) 
unique, counts = np.unique(preds, return_counts=True)
print('predict 빈도:', dict(zip(unique, counts)))

[LibSVM]predict 빈도: {-1: 2390, 1: 7610}


In [43]:
# comp_num = 2
clf = svm.OneClassSVM(nu=0.001, kernel="rbf", gamma=0.0001, verbose=True)
clf.fit(train_x[0:20000,:])
preds = clf.predict(test_x[5000:15000, :]) 
unique, counts = np.unique(preds, return_counts=True)
print('predict 빈도:', dict(zip(unique, counts)))

[LibSVM]predict 빈도: {-1: 4842, 1: 5158}


# LSTM

## model

In [5]:
class ENCODER(nn.Module):

    def __init__(self,args):
        super(ENCODER, self).__init__()
        self.args = args
        self.drop = nn.Dropout(args['dropout'])
        self.linear = nn.Linear(args['hidden_size'], args['data_dim'])

        if args['cell_type'] in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, args['cell_type'])(args['rnn_inp_size'], args['hidden_size'], args['nlayers'], dropout=args['dropout'])

    def init_weights(self):
        initrange = 0.1
        self.linear.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.fill_(0)

    def forward(self, input, hidden, return_hiddens=False, noise=False):
        output, hidden = self.rnn(input, hidden)
        output = self.linear(output.contiguous().view(-1,self.args['hidden_size']))
        output = output.contiguous().view(input.size()[0], -1, self.args['rnn_inp_size'])
        return output, hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data ############# 이게 무엇
        if self.args['cell_type'] == 'LSTM':
            return (Variable(weight.new(self.args['nlayers'], bsz, self.args['hidden_size']).zero_()),
                    Variable(weight.new(self.args['nlayers'], bsz, self.args['hidden_size']).zero_()))

    def repackage_hidden(self,h):
        """Wraps hidden states in new Variables, to detach them from their history."""
        if type(h) == tuple:
            return tuple(self.repackage_hidden(v) for v in h)
        else:
            return Variable(h.data)

    def extract_hidden(self, hidden):
        if self.args['cell_type'] == 'LSTM':
            return hidden[0][-1].data.cpu()  # hidden state last layer (hidden[1] is cell state)
        else:
            return hidden[-1].data.cpu()  # last layer

        
class DECODER(nn.Module):

    def __init__(self,args):
        super(DECODER, self).__init__()
        self.args = args
        self.drop = nn.Dropout(args['dropout'])
        self.linear = nn.Linear(args['hidden_size'], args['data_dim'])

        if args['cell_type'] in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, args['cell_type'])(args['rnn_inp_size'], args['hidden_size'], args['nlayers'], dropout=args['dropout'])


    def init_weights(self):
        initrange = 0.1
        self.linear.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.fill_(0)
        
    def forward(self, input, hidden, return_hiddens=False, noise=False):
        
        output, hidden = self.rnn(input, hidden)
        output = self.linear(output.contiguous().view(-1,self.args['hidden_size']))
        output = output.contiguous().view(input.size()[0], -1, self.args['rnn_inp_size'])

        return output, hidden




    def init_hidden(self, bsz):
        weight = next(self.parameters()).data ############# 이게 무엇
        if self.args['cell_type'] == 'LSTM':
            return (Variable(weight.new(self.args['nlayers'], bsz, self.args['hidden_size']).zero_()),
                    Variable(weight.new(self.args['nlayers'], bsz, self.args['hidden_size']).zero_()))

    def repackage_hidden(self,h):
        """Wraps hidden states in new Variables, to detach them from their history."""
        if type(h) == tuple:
            return tuple(self.repackage_hidden(v) for v in h)
        else:
            return Variable(h.data)

    def extract_hidden(self, hidden):
        if self.args['cell_type'] == 'LSTM':
            return hidden[0][-1].data.cpu()  # hidden state last layer (hidden[1] is cell state)
        else:
            return hidden[-1].data.cpu()  # last layer

## solver

In [160]:
class Solver():
    def __init__(self, args):
        
        torch.manual_seed(777)
        torch.cuda.manual_seed_all(777)
        np.random.seed(777)
        
        self.attack_list = pd.read_csv(args.attack_list_path, error_bad_lines=False, sep='\t')

        self.tf_log = args.tf_log
        
        cv = KFold(5, shuffle=False, random_state=0)


        
        train_x, test_x, test_y = dataset.lstm_dataset(train_path = args.train_path, test_path = args.test_path)
        
        cv_list = []
        cv = KFold(5, shuffle=False, random_state=0)
        for i, (idx_train, idx_test) in enumerate(cv.split(test_x)):
            cv_list.append([idx_train, idx_test])
            
        self.cv_list = cv_list      
        
        train_x_batchfy = preprocessing.batchify(args, train_x, args.batch_size)
        test_x_batchfy = preprocessing.batchify(args, test_x, args.batch_size)
        generate_batchfy = preprocessing.batchify(args, test_x, 1)
        train_generate_batchfy = preprocessing.batchify(args, train_x, 1)
        
        self.train_x_batchfy = train_x_batchfy[:,:,args.selected_dim]
        self.test_x_batchfy = test_x_batchfy[:,:,args.selected_dim]
        self.generate_batchfy = generate_batchfy[:,:,args.selected_dim]
        self.train_generate_batchfy = train_generate_batchfy[:,:,args.selected_dim]
        self.test_y = test_y
        

        self.args = args
        self.encoder = model.ENCODER(self.args)
        self.encoder.cuda()

        self.decoder = model.DECODER(self.args)
        self.decoder.cuda()

        self.optim_enc   = torch.optim.Adam(self.encoder.parameters(), self.args.lr)
        self.optim_dec   = torch.optim.Adam(self.decoder.parameters(), self.args.lr)

        self.loss_fn = nn.MSELoss()    
    
        self.logger = Logger('./tf_logs')
    
        self.base_dir = Path('model_save')
        self.base_dir.mkdir(parents=True,exist_ok=True)      
        
#         self.evaluate = args.evaluate
    
#     def make_dir_name(self, args):
#         return 'modelName:'+args.model_name+'__cellType:'+args.cell_type \
#                 + '__hidSize:' + str(args.hidden_size) + '__dropout:' + str(args.dropout)



#TODO 로그 정리
#         self.log = logging.getLogger('LSTM_log')
#         self.log.setLevel(logging.DEBUG)

#         formatter = logging.Formatter('%(asctime)s > %(message)s')

#         fileHandler = logging.FileHandler('./log/logger.txt')

#         fileHandler.setFormatter(formatter)
#         self.log.addHandler(fileHandler)
        
#         self.log.critical("Selected dim : " + str(args.selected_dim))
        log.critical("\n")
        log.critical("Selected dim : " + str(args.selected_dim))



    def load(self, path):
        try:
#             checkpoint = torch.load(Path(self.base_dir, str(args.selected_dim)))
#             checkpoint = checkpoint.with_suffix('.pth')
#             start_epoch = checkpoint['epoch
#             self.encoder.load_state_dict(checkpoint['state_dict_enc)
#             self.optim_enc.load_state_dict((checkpoint['optimizer_enc))
#             self.decoder.load_state_dict(checkpoint['state_dict_dec)
#             self.optim_dec.load_state_dict((checkpoint['optimizer_dec))            
#             del checkpoint
            print("=> loaded checkpoint")
        except:
            print("=> Not exist checkpoint")
            pass        

    def fit(self, load):
        total_loss = 0
        max_f1 = 0
        total_length = self.train_x_batchfy.size(0) - 1
        start_time = time.time()
        
        
                                          
        for epoch in range(0, self.args.epoch):

            self.encoder.train()
            self.decoder.train()
                
            hidden_enc = self.encoder.init_hidden(self.args.batch_size)

#             for batch, i in enumerate(range(0, self.train_x_batchfy.size(0) - 1, self.args.seq_length)):
#                 outSeq = []
#                 inputSeq, targetSeq = preprocessing.get_batch(self.args, self.train_x_batchfy, i)

#                 if args.seq_length != targetSeq.size()[0] :
#                     continue
#                 hidden_enc = self.encoder.repackage_hidden(hidden_enc)
#                 self.optim_enc.zero_grad()
#                 self.optim_dec.zero_grad()
                
#                 Outputseq_enc, hidden_enc = self.encoder.forward(inputSeq, hidden_enc, return_hiddens=True)
#                 deccoder_input = Variable(torch.zeros(Outputseq_enc.size())).cuda()
                
#                 deccoder_input[0,:,:] = Outputseq_enc[-1,:,:] # inputSeq[-1,:,:]
#                 deccoder_input[1:,:,:] = targetSeq[:-1,:,:]
                
#                 loss_enc = self.loss_fn(Outputseq_enc[-1,:,:].view(self.args.batch_size, -1), targetSeq[0,:,:].contiguous().view(self.args.batch_size, -1))
#                 loss_enc.backward(retain_graph=True)
                
                
#                 encoder_norm = sum(p.grad.data.abs().sum() for p in self.encoder.parameters())
                
#                 torch.nn.utils.clip_grad_norm_(self.encoder.parameters(), self.args.clip)
                
#                 self.optim_enc.step()     
                
#                 Outputseq_enc, hidden_enc = self.decoder.forward(deccoder_input, hidden_enc, return_hiddens=True)
#                 loss_dec = self.loss_fn(Outputseq_enc.view(args.batch_size, -1), targetSeq.contiguous().view(args.batch_size, -1))   
#                 loss_dec.backward()
                
#                 edecoder_norm = sum(p.grad.data.abs().sum() for p in self.decoder.parameters())
                
                
#                 torch.nn.utils.clip_grad_norm_(self.decoder.parameters(), self.args.clip)
#                 self.optim_dec.step()
                
                
                
#                 total_loss += loss_enc.item() + loss_dec.item()        

#                 if batch % 30 == 0 and self.tf_log == True :
#                     print(encoder_norm)
#                     print(decoder_norm)
#                     # 1. Log scalar values (scalar summary)
#                     info = { 'enc_loss': loss_enc.item(), 'dec_loss' : loss_dec.item() }

#                     for tag, value in info.items():
#                         self.logger.scalar_summary(tag, value, epoch*total_length + i +1)

#                     # 2. Log values and gradients of the parameters (histogram summary)
#                     for tag, value in self.encoder.named_parameters():
#                         tag = tag.replace('.', '/')
#                         self.logger.histo_summary(tag, value.data.cpu().numpy(), epoch*total_length + i +1)
                        
#                     for tag, value in self.decoder.named_parameters():
#                         tag = tag.replace('.', '/')
#                         self.logger.histo_summary(tag, value.data.cpu().numpy(), epoch*total_length + i +1)
            
            total_loss = 0    
            if len(self.args.selected_dim) == 1:
                self.anomal_score = postprocessing.get_anomalscore_encdec_1dim(base_model = self,  \
                                                                 generate_batchfy = self.generate_batchfy, length = 449916,args = self.args, cv_list = self.cv_list)
            else:    
                self.anomal_score = postprocessing.get_anomalscore_encdec(base_model = self,  \
                                                                 generate_batchfy = self.generate_batchfy, length = 449916,args = self.args, cv_list = self.cv_list)

            self.anomal_score = LA.norm(self.anomal_score, axis=1)
    #TODO conv 설정
#             max_conv, max_pre, max_recall, max_f1_tp, max_zerolist, find_attack_list = postprocessing.evaluate_conv(self.anomal_score, self.test_y, self.attack_list, 400)
    
          
#             if max_f1_tp > max_f1:
#                 end_time = time.time()
                
# #                 print("epoch[{}]\t conv[{}]\t precision[{}]\t recall[{}]\t f1[{}]\t findnum[{}]\t time[{}]".format(epoch, max_conv, max_pre, max_recall, max_f1_tp,36 - max_zerolist, end_time - start_time))
                
#                 #TODP self로 바꿔야함
#                 log.info("epoch[{}]\t conv[{}]\t precision[{}]\t recall[{}]\t f1[{}]\t findnum[{}]\t time[{}]".format(epoch, max_conv, max_pre, max_recall, max_f1_tp,36 - max_zerolist, end_time - start_time))
#                 log.info(find_attack_list)
                
#                 self.model_dictionary = {'epoch': epoch,
#                         'state_dict_enc': self.encoder.state_dict(),
#                         'optimizer_enc': self.optim_enc.state_dict(),
#                         'state_dict_dec': self.decoder.state_dict(),
#                         'optimizer_dec': self.optim_dec.state_dict(),
#                         'args':args,
#                         'loss':total_loss,
#                         'anomal_score' : self.anomal_score ,
#                          'conv' : max_conv,
#                         'f1' : max_f1_tp,
#                          'find_attack_list': find_attack_list                
#                         }
#                 self.save_checkpoint(self.args, self.model_dictionary)
#                 max_f1 = max_f1_tp
                                    
            
            
            
 
    def save_checkpoint(self, args, state):
        checkpoint = Path(self.base_dir, str(args.selected_dim))
        checkpoint = checkpoint.with_suffix('.pth')
        torch.save(state, checkpoint)
        

In [161]:
parser = argparse.ArgumentParser()

parser.add_argument("--cuda", default=True, action="store_true")
parser.add_argument("--tf_log", default=False, action="store_true")
parser.add_argument("--model_name", type=str, default="enc_dec")
parser.add_argument("--batch_size", type=int, default=256)
parser.add_argument("--clip", type=int, default=1)

parser.add_argument("--train_path", type=str, default="../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Normal_v0.csv")
parser.add_argument("--test_path", type=str, default="../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Attack_v0.csv")
parser.add_argument("--attack_list_path", type=str, default='../../DATA/SWaT/SWaT_Physical/attack_list.csv')

parser.add_argument("--dropout", type=float, default=0.5)
parser.add_argument("--hidden_size", type=int, default=128)
parser.add_argument("--nlayers", type=int, default=2)
parser.add_argument("--lr", type=float, default=0.0001)

parser.add_argument("--cell_type", type=str, default="LSTM")
parser.add_argument("--epoch", type=int, default=1)
parser.add_argument("--seq_length", type=int, default=2)
parser.add_argument('--selected_dim', nargs='+', type=int, default=[36, 38, 28, 40])
args = parser.parse_args([])

In [162]:
args.selected_dim = [0, 40]
args.hidden_size = 128
solver = Solver(args = args)

In [ ]:
import utils.postprocessing as postprocessing
solver.fit(load = False) 

torch.Size([89984, 1, 2])


In [44]:
solver.test_y[solver.cv_list[0][0]]

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')

In [ ]:
max_conv, max_pre, max_recall, max_f1_tp, max_thresholds\
,precision, recall, f1, thresholds= evaluate_conv(solver.anomal_score, solver.test_y[solver.cv_list[0][1]], solver.attack_list, 400)

In [107]:
def evaluate_conv(anomal_score, test_y, attack_list, conv):
    max_f1 = 0
    for conv in range(0, conv, 10):
        if conv != 0:
            gauss_kernel = Gaussian1DKernel(conv)
            norm = convolve(anomal_score, gauss_kernel)
        else:
            norm = anomal_score

        precision, recall, thresholds = metrics.precision_recall_curve( test_y.cpu().numpy(), norm ,pos_label =1)

        beta = 1
        f1 = (1+beta**2)*(precision*recall)/((beta**2*precision)+recall)
        f1 = np.nan_to_num(f1)

        
        
        max_pre_tp = precision[np.argmax(f1)]
        max_recall_tp = recall[np.argmax(f1)]
        max_f1_tp = f1[np.argmax(f1)]
        thresholds_tp = thresholds[np.argmax(f1)]
        
        


        if max_f1 < max_f1_tp:
            max_pre = max_pre_tp
            max_recall = max_recall_tp
            max_f1 = max_f1_tp
            max_conv = conv
            max_thresholds = thresholds_tp

#     print("conv[{}]\t precision[{}]\t recall[{}]\t f1[{}] \t find attack [{}]".format(max_conv, max_pre, max_recall, max_f1, 36 - max_zerolist))
    return max_conv, max_pre, max_recall, max_f1, max_thresholds, precision, recall, f1, thresholds

In [9]:
log = logging.getLogger('LSTM_log')
log.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s > %(message)s')

fileHandler = logging.FileHandler('./log/logger.txt')

fileHandler.setFormatter(formatter)
log.addHandler(fileHandler)

In [ ]:
args.selected_dim = [0, 40]
solver = Solver(args = args)
solver.fit(load = False) 

In [6]:
candi_1 = [0, 1, 2, 3, 4,36,37,38,39,40]
candi_2 = [0, 1, 2, 3, 4, 5]
candi_3 = [35, 36,37,38,39,40]
candi_4 = [17,18,19,20,21,22]

log = logging.getLogger('LSTM_log')
log.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s > %(message)s')

fileHandler = logging.FileHandler('./log/logger.txt')

fileHandler.setFormatter(formatter)
log.addHandler(fileHandler)
# arg_list = []    
# for i in range(len(candi_1)):
#     arg_list.append(candi_1[i])  
#     args.selected_dim = arg_list
#     solver = Solver(args = args)
#     solver.fit(load = False)
#     arg_list = []   
    
arg_list = []    
# for i in range(0,4):
# #     for k in range(0,4):
#     for p in range(0,4):
#         arg_list.append(candi_2[i])
#         arg_list.append(candi_2[i+1])
#         arg_list.append(candi_2[i+2])
#         arg_list.append(candi_3[k])
#         arg_list.append( candi_3[k+1])
#         arg_list.append( candi_3[k+2])
#         arg_list.append(candi_4[p])
#         arg_list.append(candi_4[p+1])
#         arg_list.append(candi_4[p+2])
#         args.selected_dim = arg_list
#         solver = Solver(args = args)
#         solver.fit(load = False)  
#         arg_list = []   
print("fin")

fin


In [5]:
i = 3
k =3 
p= 3
arg_list = [] 
arg_list.append(candi_2[i])
arg_list.append(candi_2[i+1])
arg_list.append(candi_2[i+2])
arg_list.append(candi_3[k])
arg_list.append( candi_3[k+1])
arg_list.append( candi_3[k+2])
arg_list.append(candi_4[p])
arg_list.append(candi_4[p+1])
arg_list.append(candi_4[p+2])
args.selected_dim = arg_list
solver = Solver(args = args)
solver.fit(load = False)  

In train : {'Normal': 496800}
In test : {'Attack': 54621, 'Normal': 395298}
data length is 51


In [4]:
whole = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
args.selected_dim = whole
solver = Solver(args = args)
solver.fit(load = False)



In train : {'Normal': 496800}
In test : {'Attack': 54621, 'Normal': 395298}
data length is 51
epoch[0]	 conv[190]	 precision[0.4968557953408604]	 recall[0.6364768129474012]	 f1[0.5580659919255805]	 findnum[25]	 time[232.773029088974]
epoch[2]	 conv[190]	 precision[0.6496848335767546]	 recall[0.6189377711869062]	 f1[0.6339387006947506]	 findnum[23]	 time[694.000387430191]
epoch[4]	 conv[180]	 precision[0.6844791962079164]	 recall[0.6186265355815529]	 f1[0.6498889284237452]	 findnum[21]	 time[1156.1872382164001]
epoch[5]	 conv[190]	 precision[0.7115196842214605]	 recall[0.6270298969260907]	 f1[0.6666082759157616]	 findnum[21]	 time[1386.4979054927826]
epoch[7]	 conv[190]	 precision[0.8244008427706083]	 recall[0.6877025319931894]	 f1[0.749872735439437]	 findnum[21]	 time[1848.0769476890564]
epoch[8]	 conv[140]	 precision[0.7867742238063972]	 recall[0.7678182384064737]	 f1[0.7771806608231566]	 findnum[21]	 time[2078.4752616882324]
